In [5]:
import heapq
import numpy as np
from collections import defaultdict

# Given data
vertices = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# graph
edges = defaultdict(dict)
edges[1][2] = 1
edges[1][6] = 1
edges[2][1] = 1
edges[2][5] = 1
edges[2][3] = 1
edges[3][2] = 1
edges[3][4] = 1
edges[4][3] = 1
edges[4][5] = 1
edges[5][2] = 1
edges[5][4] = 1
edges[5][6] = 1
edges[6][1] = 1
edges[6][5] = 1
edges[6][7] = 1
edges[7][6] = 1
edges[7][8] = 1
edges[8][7] = 1
edges[8][9] = 1
edges[9][8] = 1
edges[9][10] = 1
edges[10][9] = 1

# heuristic
heuristic = {}
heuristic[1] = 5
heuristic[2] = 4
heuristic[3] = 3
heuristic[4] = 2
heuristic[5] = 3
heuristic[6] = 4
heuristic[7] = 3
heuristic[8] = 2
heuristic[9] = 1
heuristic[10] = 0

class Node(object):
    def __init__(self, key):
        self.key = key
        self.parent = None
        self.f = float('inf')
        self.g = float('inf')
        self.h = 0.0
        self.is_open = False
        self.is_closed = False
        self.children = []

    def setParent(self, parent):
        self.parent = parent

    def setChildren(self, children):
        self.children = children

    def setG(self, g):
        self.g = g
        self.f = self.g + self.h

    def setHeuristic(self, h):
        self.h = h
        self.f = self.g + self.h

    def getHeuristic(self):
        return self.h

    def isOpen(self):
        return self.is_open

    def isClosed(self):
        return self.is_closed

    def getParent(self):
        return self.parent

    def getG(self):
        return self.g

    def getF(self):
        self.f = self.g + self.h
        return self.f

def init_nodes(start, heuristic):
    open_heap = []
    open_set = {}
    nodes = {}
    for v in vertices:
        node = Node(v)
        node.setHeuristic(heuristic[v])
        node.setChildren(list(edges[v].keys()))
        if v == start:
            node.setG(0)
            node.is_open = True
            f = node.getF()
            heapq.heappush(open_heap, (f, v))
            open_set[v] = f
        nodes[v] = node
    return open_heap, open_set, nodes

def find_node(nodes, key):
    return nodes.get(key)

def find_optimal(open_set):
    if not open_set:
        return None, float('inf')
    return min(open_set.items(), key=lambda item: item[1])

def update_heuristic(closed_list, heuristic, node_dict, f):
    for v in closed_list:
        node = node_dict[v]
        heuristic[v] = f - node.getG()

def get_path(optimal_key, node_dict):
    path = []
    current_node = node_dict[optimal_key]

    while current_node is not None:
        path.append(current_node.key)
        current_node = current_node.getParent()
    return list(reversed(path))

def a_star(start, goal, heuristic, step=4):
    open_heap, open_set, node_dict = init_nodes(start, heuristic)
    closed_list = []

    # Reset the nodes for a fresh search
    for key, node in node_dict.items():
        if key != start:
            node.setG(float('inf'))
        node.parent = None
        node.is_open = False
        node.is_closed = False

    start_node = node_dict[start]
    start_node.setG(0)
    start_node.is_open = True

    expanded_count = 0

    while open_heap and expanded_count < step:
        # Get the node with the lowest f-score
        current_f, current_key = heapq.heappop(open_heap)

        # Check if this node is still in the open set with the same f-score
        if current_key not in open_set or open_set[current_key] != current_f:
            continue

        # Remove from open set
        del open_set[current_key]

        current_node = node_dict[current_key]
        current_node.is_open = False
        current_node.is_closed = True
        closed_list.append(current_key)

        # Check if goal reached
        if current_key == goal:
            return open_set, closed_list, node_dict, True

        # expand node
        for child_key, edge_cost in edges[current_key].items():
            child_node = node_dict[child_key]
            if child_node.is_closed:
                continue

            tentative_g = current_node.getG() + edge_cost
            if tentative_g < child_node.getG():
                child_node.setParent(current_node)
                child_node.setG(tentative_g)

                # update
                f = child_node.getF()
                if child_key not in open_set or f < open_set[child_key]:
                    heapq.heappush(open_heap, (f, child_key))
                    open_set[child_key] = f
                    child_node.is_open = True

        expanded_count += 1

    return open_set, closed_list, node_dict, False

def rtaa_star(start, goal, heuristic, step=4):
    path = [start]
    current = start
    iteration = 0

    while current != goal:
        print("========================")
        print(f"Iteration {iteration + 1}: Current position = {current}")

        # expand by A* for a limited number of steps
        open_set, closed_list, node_dict, goal_reached = a_star(current, goal, heuristic, step)

        if goal_reached:
            # complete the path to the goal
            remaining_path = get_path(goal, node_dict)[1:]
            path.extend(remaining_path)
            print(f"Goal reached Final path: {path}")
            return path

        if not open_set:
            print("Failed to find a path. No nodes in open set.")
            return path

        # find the best next node to move to
        optimal_key, optimal_cost = find_optimal(open_set)
        if optimal_key is None:
            print("No optimal node found. Path finding failed.")
            return path

        # update heuristics for closed nodes
        update_heuristic(closed_list, heuristic, node_dict, optimal_cost)
        print(f"CLOSED list: {closed_list}")
        print(f"OPEN list: {open_set}")
        print(f"HEURISTIC: {heuristic}")

        # move to the next best node
        next_node = optimal_key
        next_path = get_path(next_node, node_dict)
        if len(next_path) > 1:
            move_segment = next_path[1:]
            path.extend(move_segment)
            current = next_node
            print(f"Moving to node {current}, path segment: {move_segment}")
        else:
            print("No valid move found. Path finding failed.")
            return path
        iteration += 1
    return path

print("Starting RTAA* algorithm")
final_path = rtaa_star(1, 10, heuristic.copy(), step=4)
#print(f"Final path from 1 to 10: {final_path}")

Starting RTAA* algorithm
Iteration 1: Current position = 1
CLOSED list: [1, 2, 3, 4]
OPEN list: {6: 5, 5: 5}
HEURISTIC: {1: 5, 2: 4, 3: 3, 4: 2, 5: 3, 6: 4, 7: 3, 8: 2, 9: 1, 10: 0}
Moving to node 6, path segment: [6]
Iteration 2: Current position = 6
CLOSED list: [6, 5, 4, 7]
OPEN list: {1: 6, 2: 6, 3: 6, 8: 4}
HEURISTIC: {1: 5, 2: 4, 3: 3, 4: 2, 5: 3, 6: 4, 7: 3, 8: 2, 9: 1, 10: 0}
Moving to node 8, path segment: [7, 8]
Iteration 3: Current position = 8
Goal reached Final path: [1, 6, 7, 8, 9, 10]


In [6]:

x = []
heapq.heappush(x, (1, 3))
heapq.heappush(x, (1, 5))
current_f, current_key = heapq.heappop(x)
print(current_key)

3
